<a href="https://colab.research.google.com/github/ktam069/Google_Colab_Audiovisual/blob/master/Timing_v2_modified_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Timing speed of bill v2 on CPU

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls '/content/gdrive/My Drive/Part IV Project/Colab Modified/test_inputs/'

cRMs_output_00000-00011-.npy  mix_00001-00005-.npy  mix_00005-00009-.npy
face_embs_00000-00011-.npy    mix_00001-00006-.npy  mix_00005-00010-.npy
face_embs_00000-00019-.npy    mix_00001-00007-.npy  mix_00005-00011-.npy
face_embs_00001-00015-.npy    mix_00001-00008-.npy  mix_00005-00014-.npy
face_embs_00001-00019-.npy    mix_00001-00009-.npy  mix_00005-00015-.npy
face_embs_00003-00015-.npy    mix_00001-00010-.npy  mix_00005-00016-.npy
face_embs_00004-00009-.npy    mix_00001-00011-.npy  mix_00005-00018-.npy
face_embs_00007-00010-.npy    mix_00001-00014-.npy  mix_00005-00019-.npy
face_embs_00007-00015-.npy    mix_00001-00015-.npy  mix_00006-00009-.npy
face_embs_00007-00016-.npy    mix_00001-00016-.npy  mix_00006-00010-.npy
face_embs_00008-00014-.npy    mix_00001-00018-.npy  mix_00006-00011-.npy
face_embs_00008-00016-.npy    mix_00001-00019-.npy  mix_00006-00014-.npy
max_expand_00000-00011-.npy   mix_00003-00005-.npy  mix_00006-00015-.npy
max_expand_00000-00019-.npy   mix_00003-00006-.npy 

In [0]:
!cp -r '/content/gdrive/My Drive/Part IV Project/Colab Modified/test_inputs/' "test_inputs/"

In [4]:
!ls

gdrive	sample_data  test_inputs


In [5]:
!ls "test_inputs/"

cRMs_output_00000-00011-.npy  mix_00001-00005-.npy  mix_00005-00009-.npy
face_embs_00000-00011-.npy    mix_00001-00006-.npy  mix_00005-00010-.npy
face_embs_00000-00019-.npy    mix_00001-00007-.npy  mix_00005-00011-.npy
face_embs_00001-00015-.npy    mix_00001-00008-.npy  mix_00005-00014-.npy
face_embs_00001-00019-.npy    mix_00001-00009-.npy  mix_00005-00015-.npy
face_embs_00003-00015-.npy    mix_00001-00010-.npy  mix_00005-00016-.npy
face_embs_00004-00009-.npy    mix_00001-00011-.npy  mix_00005-00018-.npy
face_embs_00007-00010-.npy    mix_00001-00014-.npy  mix_00005-00019-.npy
face_embs_00007-00015-.npy    mix_00001-00015-.npy  mix_00006-00009-.npy
face_embs_00007-00016-.npy    mix_00001-00016-.npy  mix_00006-00010-.npy
face_embs_00008-00014-.npy    mix_00001-00018-.npy  mix_00006-00011-.npy
face_embs_00008-00016-.npy    mix_00001-00019-.npy  mix_00006-00014-.npy
max_expand_00000-00011-.npy   mix_00003-00005-.npy  mix_00006-00015-.npy
max_expand_00000-00019-.npy   mix_00003-00006-.npy 

In [6]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print(tf.__version__)

1.14.0


In [0]:
'''Load one entry of the dataset'''
mix_expand_path = "test_inputs/" + "max_expand_00000-00011-.npy"
face_embs_path = "test_inputs/" + "face_embs_00000-00011-.npy"
mix_path = "test_inputs/" + "mix_00000-00011-.npy"

mix_expand = np.load(mix_expand_path)
face_embs = np.load(face_embs_path)
mix = np.load(mix_path)

In [8]:
print(mix_expand.shape)
print(face_embs.shape)
print(mix.shape)

(1, 298, 257, 2)
(1, 75, 1, 1792, 2)
(298, 257, 2)


In [9]:
dataset = [mix_expand, face_embs]
#dataset = np.array(dataset)
#print(dataset.shape)
print(len(dataset))
print(len(dataset[0][0]))
print(len(dataset[1][0]))

2
298
75


In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab TFServing/AVmodel-2p-006-0.75753.h5' "AVmodel-2p-006-0.75753.h5"

In [0]:
def audio_discriminate_loss2(gamma=0.1,beta = 2*0.1,num_speaker=2):
    def loss_func(S_true,S_pred,gamma=gamma,beta=beta,num_speaker=num_speaker):
        import keras.backend as K
        sum_mtr = K.zeros_like(S_true[:,:,:,:,0])
        for i in range(num_speaker):
            sum_mtr += K.square(S_true[:,:,:,:,i]-S_pred[:,:,:,:,i])
            for j in range(num_speaker):
                if i != j:
                    sum_mtr -= gamma*(K.square(S_true[:,:,:,:,i]-S_pred[:,:,:,:,j]))

        for i in range(num_speaker):
            for j in range(i+1,num_speaker):
                #sum_mtr -= beta*K.square(S_pred[:,:,:,i]-S_pred[:,:,:,j])
                #sum_mtr += beta*K.square(S_true[:,:,:,:,i]-S_true[:,:,:,:,j])
                pass
        #sum = K.sum(K.maximum(K.flatten(sum_mtr),0))

        loss = K.mean(K.flatten(sum_mtr))

        return loss
    return loss_func

audio_loss = audio_discriminate_loss2

In [0]:
# PATH
# model_path = './AVmodel-2p-002-0.87080.h5'	# 234K
model_path = './AVmodel-2p-006-0.75753.h5'	# 50K

In [0]:
from tensorflow.keras.models import load_model

In [15]:
'''Load model'''
# super parameters
people_num = 2
epochs = 100
initial_epoch = 0
batch_size = 2 # 4 to feed one 16G GPU
gamma_loss = 0.1
beta_loss = gamma_loss*2

loss_func = audio_loss(gamma=gamma_loss,beta=beta_loss, num_speaker=people_num)
#model = load_model(model_path)
model = load_model(model_path,custom_objects={"tf": tf, 'loss_func': loss_func})


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [0]:
import numpy as np
import librosa
import os 
from tensorflow.python.framework import tensor_util
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.core.framework import graph_pb2
import operator
import itertools
from numpy import inf

#### Load Data ####
# windowing fft/ifft function
def stft(data, fft_size=512, step_size=160,padding=True):
    # short time fourier transform
    if padding == True:
        # for 16K sample rate data, 48192-192 = 48000
        pad = np.zeros(192,)
        data = np.concatenate((data,pad),axis=0)
    # padding hanning window 512-400 = 112
    window = np.concatenate((np.zeros((56,)),np.hanning(fft_size-112),np.zeros((56,))),axis=0)
    win_num = (len(data) - fft_size) // step_size
    out = np.ndarray((win_num, fft_size), dtype=data.dtype)
    for i in range(win_num):
        left = int(i * step_size)
        right = int(left + fft_size)
        out[i] = data[left: right] * window
    F = np.fft.rfft(out, axis=1)
    return F

def istft(F, fft_size=512, step_size=160,padding=True):
    # inverse short time fourier transform
    data = np.fft.irfft(F, axis=-1)
    # padding hanning window 512-400 = 112
    window = np.concatenate((np.zeros((56,)),np.hanning(fft_size-112),np.zeros((56,))),axis=0)
    number_windows = F.shape[0]
    T = np.zeros((number_windows * step_size + fft_size))
    for i in range(number_windows):
        head = int(i * step_size)
        tail = int(head + fft_size)
        T[head:tail] = T[head:tail] + data[i, :] * window
    if padding == True:
        T = T[:48000]
    return T

# combine FFT bins to mel frequency bins
def mel2freq(mel_data,sr,fft_size,n_mel,fmax=8000):
    matrix= librosa.filters.mel(sr, fft_size, n_mel, fmax=fmax)
    return np.dot(mel_data,matrix)

def freq2mel(f_data,sr,fft_size,n_mel,fmax=8000):
    pre_matrix = librosa.filters.mel(sr, fft_size, n_mel, fmax=fmax)
    matrix = pre_matrix.T / np.sum(pre_matrix.T,axis=0)
    return np.dot(f_data,matrix)

# directly time to mel domain transformation
def time_to_mel(data,sr,fft_size,n_mel,step_size,fmax=8000):
    F = stft(data,fft_size,step_size)
    M = freq2mel(F,sr,fft_size,n_mel,fmax=8000)
    return M

def mel_to_time(M,sr,fft_size,n_mel,step_size,fmax=8000):
    F = mel2freq(M,sr,fft_size,n_mel)
    T = istft(F,fft_size,step_size)
    return T

def real_imag_expand(c_data,dim='new'):
    # dim = 'new' or 'same'
    # expand the complex data to 2X data with true real and image number
    if dim == 'new':
        D = np.zeros((c_data.shape[0],c_data.shape[1],2))
        D[:,:,0] = np.real(c_data)
        D[:,:,1] = np.imag(c_data)
        return D
    if dim =='same':
        D = np.zeros((c_data.shape[0],c_data.shape[1]*2))
        D[:,::2] = np.real(c_data)
        D[:,1::2] = np.imag(c_data)
        return D

def real_imag_shrink(F,dim='new'):
    # dim = 'new' or 'same'
    # shrink the complex data to combine real and imag number
    F_shrink = np.zeros((F.shape[0], F.shape[1]))
    if dim =='new':
        F_shrink = F[:,:,0] + F[:,:,1]*1j
    if dim =='same':
        F_shrink = F[:,::2] + F[:,1::2]*1j
    return F_shrink

def power_law(data,power=0.6):
    # assume input has negative value
    mask = np.zeros(data.shape)
    mask[data>=0] = 1
    mask[data<0] = -1
    data = np.power(np.abs(data),power)
    data = data*mask
    return data

def fast_stft(data,power=False,**kwargs):
    # directly transform the wav to the input
    # power law = A**0.3 , to prevent loud audio from overwhelming soft audio
    if power:
        data = power_law(data)
    return real_imag_expand(stft(data))

def fast_istft(F,power=False,**kwargs):
    # directly transform the frequency domain data to time domain data
    # apply power law
    T = istft(real_imag_shrink(F))
    if power:
        T = power_law(T,(1.0/0.6))
    return T


def generate_cRM(Y,S):
    '''

    :param Y: mixed/noisy stft
    :param S: clean stft
    :return: structed cRM
    '''
    M = np.zeros(Y.shape)
    epsilon = 1e-8
    # real part
    M_real = np.multiply(Y[:,:,0],S[:,:,0])+np.multiply(Y[:,:,1],S[:,:,1])
    square_real = np.square(Y[:,:,0])+np.square(Y[:,:,1])
    M_real = np.divide(M_real,square_real+epsilon)
    M[:,:,0] = M_real
    # imaginary part
    M_img = np.multiply(Y[:,:,0],S[:,:,1])-np.multiply(Y[:,:,1],S[:,:,0])
    square_img = np.square(Y[:,:,0])+np.square(Y[:,:,1])
    M_img = np.divide(M_img,square_img+epsilon)
    M[:,:,1] = M_img
    return M

def cRM_tanh_compress(M,K=10,C=0.1):
    '''
    Recall that the irm takes on vlaues in the range[0,1],compress the cRM with hyperbolic tangent
    :param M: crm (298,257,2)
    :param K: parameter to control the compression
    :param C: parameter to control the compression
    :return crm: compressed crm
    '''

    numerator = 1-np.exp(-C*M)
    numerator[numerator == inf] = 1
    numerator[numerator == -inf] = -1
    denominator = 1+np.exp(-C*M)
    denominator[denominator == inf] = 1
    denominator[denominator == -inf] = -1
    crm = K * np.divide(numerator,denominator)

    return crm

def cRM_tanh_recover(O,K=10,C=0.1):
    '''

    :param O: predicted compressed crm
    :param K: parameter to control the compression
    :param C: parameter to control the compression
    :return M : uncompressed crm
    '''

    numerator = K-O
    denominator = K+O
    M = -np.multiply((1.0/C),np.log(np.divide(numerator,denominator)))

    return M


def fast_cRM(Fclean,Fmix,K=10,C=0.1):
    '''

    :param Fmix: mixed/noisy stft
    :param Fclean: clean stft
    :param K: parameter to control the compression
    :param C: parameter to control the compression
    :return crm: compressed crm
    '''
    M = generate_cRM(Fmix,Fclean)
    crm = cRM_tanh_compress(M,K,C)
    return crm

def fast_icRM(Y,crm,K=10,C=0.1):
    '''
    :param Y: mixed/noised stft
    :param crm: DNN output of compressed crm
    :param K: parameter to control the compression
    :param C: parameter to control the compression
    :return S: clean stft
    '''
    M = cRM_tanh_recover(crm,K,C)
    S = np.zeros(np.shape(M))
    S[:,:,0] = np.multiply(M[:,:,0],Y[:,:,0])-np.multiply(M[:,:,1],Y[:,:,1])
    S[:,:,1] = np.multiply(M[:,:,0],Y[:,:,1])+np.multiply(M[:,:,1],Y[:,:,0])
    return S

In [0]:
prefix = "--"
dir_path = './pred/'
if not os.path.isdir(dir_path):
	os.mkdir(dir_path)

import time

In [0]:
import scipy.io.wavfile as wavfile

AV_model = model

In [19]:
num_tests = 30

times = []
post_processing_times = []

for i in range(num_tests):
	'''Predict data'''
	start_time = time.time()
	cRMs = AV_model.predict([mix_expand, face_embs])
	end_time = time.time()
	runtime = end_time-start_time
	times.append(runtime)
	print("Time taken for run predict %d:"%(i+1), runtime)

	'''Save output as wav'''
	start_time = time.time()
	cRMs = cRMs[0]
	for j in range(people_num):
		cRM = cRMs[:,:,:,j]
		assert cRM.shape == (298,257,2)
		F = fast_icRM(mix,cRM)
		T = fast_istft(F,power=False)
		filename = dir_path+prefix+'.wav'
		wavfile.write(filename,16000,T)

	end_time = time.time()
	runtime = end_time-start_time
	post_processing_times.append(runtime)

	print("Time taken to post-process %d:"%(i+1), runtime)

Time taken for run predict 1: 14.84204912185669
Time taken to post-process 1: 0.021223068237304688
Time taken for run predict 2: 13.990865707397461
Time taken to post-process 2: 0.020837783813476562
Time taken for run predict 3: 13.427454233169556
Time taken to post-process 3: 0.02190089225769043
Time taken for run predict 4: 12.513241052627563
Time taken to post-process 4: 0.021254777908325195
Time taken for run predict 5: 12.90308427810669
Time taken to post-process 5: 0.019403934478759766
Time taken for run predict 6: 13.068732976913452
Time taken to post-process 6: 0.01705336570739746
Time taken for run predict 7: 13.455395936965942
Time taken to post-process 7: 0.02019667625427246
Time taken for run predict 8: 12.575503826141357
Time taken to post-process 8: 0.017670869827270508
Time taken for run predict 9: 12.395727396011353
Time taken to post-process 9: 0.020185232162475586
Time taken for run predict 10: 12.20577359199524
Time taken to post-process 10: 0.017850160598754883
Time

In [20]:
list_of_time = times

avg = sum(list_of_time) / len(list_of_time)
print("Number of test:", len(list_of_time))
print("Test times:", list_of_time)
print("Average running time:", avg)

Number of test: 30
Test times: [14.84204912185669, 13.990865707397461, 13.427454233169556, 12.513241052627563, 12.90308427810669, 13.068732976913452, 13.455395936965942, 12.575503826141357, 12.395727396011353, 12.20577359199524, 12.659483909606934, 12.099591493606567, 12.164552927017212, 12.280062675476074, 12.53912901878357, 12.193217039108276, 12.203923463821411, 12.534335613250732, 14.050617694854736, 14.22521710395813, 14.282251834869385, 14.156654119491577, 13.77782154083252, 14.196676015853882, 14.21888279914856, 14.174191236495972, 14.267903327941895, 13.771976470947266, 13.687594413757324, 12.446892261505127]
Average running time: 13.243626769383749


In [21]:
list_of_time = post_processing_times

avg = sum(list_of_time) / len(list_of_time)
print("Number of test:", len(list_of_time))
print("Test times:", list_of_time)
print("Average running time:", avg)

Number of test: 30
Test times: [0.021223068237304688, 0.020837783813476562, 0.02190089225769043, 0.021254777908325195, 0.019403934478759766, 0.01705336570739746, 0.02019667625427246, 0.017670869827270508, 0.020185232162475586, 0.017850160598754883, 0.019469261169433594, 0.018959999084472656, 0.01752614974975586, 0.018616914749145508, 0.017851829528808594, 0.021512985229492188, 0.01800251007080078, 0.020791053771972656, 0.022852659225463867, 0.02120685577392578, 0.021762847900390625, 0.02064824104309082, 0.02031874656677246, 0.023659944534301758, 0.02174997329711914, 0.02256631851196289, 0.022552013397216797, 0.018959522247314453, 0.02204751968383789, 0.017835140228271484]
Average running time: 0.020215574900309244


In [0]:
np.savetxt("Cloud_modifiedv2_times_CPU.csv", times, delimiter=",")

In [0]:
np.savetxt("Cloud_modifiedv2_postpro_times_CPU.csv", post_processing_times, delimiter=",")

In [0]:
new_dir = "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_speech_output/"
if not os.path.isdir(new_dir):
  os.mkdir(new_dir)

In [0]:
!cp "Cloud_modifiedv2_times_CPU.csv" "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_speech_output/Cloud_modifiedv2_times_CPU.csv"

In [0]:
!cp "Cloud_modifiedv2_postpro_times_CPU.csv" "/content/gdrive/My Drive/Part IV Project/Colab TFServing/AV_speech_output/Cloud_modifiedv2_postpro_times_CPU.csv"

In [27]:
AV_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 298, 257, 2) 0                                            
__________________________________________________________________________________________________
as_conv1 (Conv2D)               (None, 298, 257, 96) 1440        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 298, 257, 96) 384         as_conv1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 298, 257, 96) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

Most of the following code in this notebook was adapted from https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple

In [28]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')
  !rm -r {export_path}

tf.saved_model.simple_save(
    keras.backend.get_session(),
    export_path,
    inputs={'input_image': model.input},
    outputs={t.name:t for t in model.outputs})

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


AttributeError: ignored

In [0]:
print('\nSaved model:')
!ls -l {export_path}

In [0]:
!saved_model_cli show --dir {export_path} --all